# 1.Diff-in-Diff & NLP_Feature_Engineering Overview

In order to investigate on seasonal effect on visitors' sentiment changing. We define 3 seasons' diff-in-diff tests, 
Define City Groups: Barcelona is City group = 1, and Florence is base group = 0.

Define Pre- and Post-Treatment Periods: We want to compare the periods March to June, June to September, and September to January. And We consider May, August, and December as the months immediately before the treatment effect.

Compute the DiD Estimator: Compute the average change in the outcome variable (Sentiment Score) for the treatment and control groups before and after the treatment. The DiD estimator is the difference between these changes.

### Step 1. Analysis Diff-in-Diff directly
We directly apply diff-in-diff without specify text one-hot encoding
### Step 2. Analysis Diff-in-Diff after adding one-hot encoding(nlp feature engineering)
After apply tf-idf, we analyze diff-in-diff again. Suggest to have better result, as text one-hot should remove somehow confounder effects.

In [ ]:
#!pip install dtaidistance 
#!pip install stargazer

###################################!pip install --upgrade pip################# upgrade pip ###############################
#!pip install gensim --verbose

In [12]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surface\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
import os

import pandas as pd
import numpy as np
from dtaidistance import dtw

import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.display import HTML

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

#!python -m spacy download en_core_web_sm
import spacy

import re
from collections import Counter

from tqdm import tqdm
tqdm.pandas()

## 2. Directly Diff-in-Diff

Define City Groups: Barcelona is City group = 1, and Florence is base group = 0.

Define Pre- and Post-Treatment Periods: We want to compare the periods March to June, June to September, and September to January. And We consider May, August, and December as the months immediately before the treatment effect.

Compute the DiD Estimator: Compute the average change in the outcome variable (Sentiment Score) for the treatment and control groups before and after the treatment. The DiD estimator is the difference between these changes.

In [15]:
# Specify file path and name
save_sentiment_scored_and_filtered_from_2023_to_2034_Barcelona_Paris_file_path = 'C:/Users/surface/DSDM/Semesters/semester2/text mining/Final Project/nlp_data/Step_1st_nlp_sentiment_scored_and_filtered_from_2023_to_2034_Barcelona_Paris_file_path.csv'

# modify format
# Save the combined DataFrame to a CSV file
df_copy = pd.read_csv(save_sentiment_scored_and_filtered_from_2023_to_2034_Barcelona_Paris_file_path, encoding='utf-8')  # `index=False` to avoid writing row indices to the file
df_copy.head()

,Reviewer_Name,Review_Body,Review_Date,Rating,Country,Helpful_Tag,Travel Category,Require_Translate,City,Attraction_Rank,Is_Commented,Sentiment_Score,Year_Month
0,Sylvie,No welcome and self-guided support🥴,2023-01-01,3,France,Helpful,NaN,True,Barcelona,top1,True,0.056700,2023-01
1,Dobromir,There are no comments available for this review,2023-01-01,4,Bulgaria,NaN,NaN,False,Barcelona,top1,False,0.506917,2023-01
2,Paolo,Audio guide not present and not explained,2023-01-02,3,Switzerland,Helpful,NaN,True,Barcelona,top1,True,0.000000,2023-01
3,Fadhil,Too many steps to get only to the ground floor...,2023-01-02,2,Switzerland,Helpful,NaN,True,Barcelona,top1,True,0.636100,2023-01
4,Brunella,There are no comments available for this review,2023-01-02,5,Italy,NaN,NaN,False,Barcelona,top1,False,0.506917,2023-01


In [ ]:
def create_season_variables(df):
    # Define seasons and corresponding months
    spring = ['March', 'April', 'May', 'June']
    summer = ['June', 'July', 'August', 'September']
    fall = ['September', 'October', 'November', 'December']
    
    # Define destination cities and their groups
    destination_cities = {'Florence': 0, 'Barcelona': 1}
    
    # Copy the DataFrame to avoid modifying the original
    df_copy = df.copy()
    
    # Convert 'Review_Date' to datetime and create a new column for year and month
    df_copy['Review_Date'] = pd.to_datetime(df_copy['Review_Date'])
    df_copy['Year_Month'] = df_copy['Review_Date'].dt.to_period('M')
    
    # Create a new column for the city group
    df_copy['City_Group'] = df_copy['City'].map(destination_cities)
    df_copy.dropna(subset=['City_Group'], inplace=True)  # drop not selected city
    
    # Initialize 'Treatment' and 'Diff_Estimator' columns
    df_copy['Treatment'] = 0
    df_copy['Diff_Estimator'] = 0

    # Set 'Treatment' column based on the specified treatment months within each season
    treatment_months = {'spring': ['May'], 'summer': ['August'], 'fall': ['December']}
    for season, months in treatment_months.items():
        treatment_index = df_copy[df_copy['Year_Month'].dt.strftime('%B').isin(months)].index
        df_copy.loc[treatment_index, 'Treatment'] = 1
    
    # Convert 'City_Group', 'Treatment', and 'Diff_Estimator' to integer type
    df_copy['City_Group'] = df_copy['City_Group'].astype(int)
    df_copy['Treatment'] = df_copy['Treatment'].astype(int)
    #df_copy['Diff_Estimator'] = df_copy['Diff_Estimator'].astype(int)

    # Split the DataFrame into seasons
    spring_data = df_copy[df_copy['Year_Month'].dt.strftime('%B').isin(spring)].copy()
    summer_data = df_copy[df_copy['Year_Month'].dt.strftime('%B').isin(summer)].copy()
    fall_data = df_copy[df_copy['Year_Month'].dt.strftime('%B').isin(fall)].copy()
    
    # Set months after treatment to 1 within each season
    for season_data in [spring_data, summer_data, fall_data]:
        treatment_indices = season_data[season_data['Treatment'] == 1].index
        if len(treatment_indices) > 0:
            season_data.loc[treatment_indices[0]:, 'Treatment'] = 1
        # Diff_estimator = Treatment x City_Group
        season_data['Diff_Estimator'] = season_data['Treatment'] * season_data['City_Group']
    # Compute the mean sentiment score for each month within each season
    spring_monthly_sentiment = spring_data.groupby(['Year_Month','City_Group', 'Treatment', 'Diff_Estimator'])['Sentiment_Score'].mean().reset_index()
    summer_monthly_sentiment = summer_data.groupby(['Year_Month','City_Group', 'Treatment', 'Diff_Estimator'])['Sentiment_Score'].mean().reset_index()
    fall_monthly_sentiment = fall_data.groupby(['Year_Month','City_Group', 'Treatment', 'Diff_Estimator'])['Sentiment_Score'].mean().reset_index()
    
    return spring_monthly_sentiment, summer_monthly_sentiment, fall_monthly_sentiment

# Create seasonal variables and compute mean sentiment score for each season
spring_monthly_sentiment, summer_monthly_sentiment, fall_monthly_sentiment = create_season_variables(df_copy)

In [ ]:
display(spring_monthly_sentiment.head()),display(summer_monthly_sentiment.head()),display(fall_monthly_sentiment.head())

,Year_Month,City_Group,Treatment,Diff_Estimator,Sentiment_Score
0,2023-03,0,1,0,0.655512
1,2023-03,1,0,0,0.492363
2,2023-04,0,1,0,0.679126
3,2023-04,1,0,0,0.499612
4,2023-05,0,1,0,0.614473
5,2023-05,1,1,1,0.476266
6,2023-06,0,1,0,0.606129
7,2023-06,1,1,1,0.461799


,Year_Month,City_Group,Treatment,Diff_Estimator,Sentiment_Score
0,2023-06,0,1,0,0.606129
1,2023-06,1,0,0,0.461799
2,2023-07,0,1,0,0.592441
3,2023-07,1,0,0,0.481961
4,2023-08,0,1,0,0.556159
5,2023-08,1,1,1,0.467471
6,2023-09,0,1,0,0.586094
7,2023-09,1,1,1,0.495672


,Year_Month,City_Group,Treatment,Diff_Estimator,Sentiment_Score
0,2023-09,0,1,0,0.586094
1,2023-09,1,0,0,0.495672
2,2023-10,0,1,0,0.594617
3,2023-10,1,0,0,0.471328
4,2023-11,0,1,0,0.601568
5,2023-11,1,0,0,0.474616
6,2023-12,0,1,0,0.539333
7,2023-12,1,1,1,0.476965


(None, None, None)

In [ ]:
'''def compute_monthly_sentiment(season_data):
    # Group by both season and month, then calculate the mean sentiment score for each group
    monthly_sentiment = season_data.groupby(['Year_Month'])['Sentiment_Score'].mean().reset_index()
    
    return monthly_sentiment'''

"def compute_monthly_sentiment(season_data):\n    # Group by both season and month, then calculate the mean sentiment score for each group\n    monthly_sentiment = season_data.groupby(['Year_Month'])['Sentiment_Score'].mean().reset_index()\n    \n    return monthly_sentiment"

In [ ]:
def apply_diff_in_diff(season_data, feature_engineering = None):
    # Define the regression model
    model1 = sm.OLS(endog=season_data['Sentiment_Score'], 
                   exog=sm.add_constant(season_data[['Treatment']]))
    model2 = sm.OLS(endog=season_data['Sentiment_Score'], 
                   exog=sm.add_constant(season_data[['City_Group']]))
    model3 = sm.OLS(endog=season_data['Sentiment_Score'], 
                   exog=sm.add_constant(season_data[['Treatment', 'City_Group', 'Diff_Estimator']]))
    if(feature_engineering is not None):
        features = ['Treatment', 'City_Group', 'Diff_Estimator']
        features.extend(feature_engineering)
        model4 = sm.OLS(endog=season_data['Sentiment_Score'], 
                   exog=sm.add_constant(season_data[features]))
    else:
        model4 = None
    
    # Fit the model
    results1 = model1.fit()
    results2 = model2.fit()
    results3 = model3.fit()
    if model4 is not None:
        results4 = model4.fit()
    else:
        results4 = None
    
    return results1, results2, results3, results4 if results4 is not None else None

### Apply DiD for each season

#### Spring to Summer: event on May

In [ ]:
spr_results1, spr_results2, spr_results3, _ = apply_diff_in_diff(spring_monthly_sentiment)

stargazer = Stargazer([spr_results1, spr_results2, spr_results3])
HTML(stargazer.render_html())

Base: Florence = 0, Bacelona = 1<br>

In Spring to Summer season. May is event.<br>

Coefficient analysis: most significant at 0.01<br>
Event Treatment: showing a 0.086 increasing trend over all reviews sentiment. But not significant<br>
City: -0.156 showing a decreasing trend compare Barcelona to Florence. Significant<br>
Diff-in-Diff estimator: Showing a -0.216 decreasing trend for Barcelona, after event happened.<br>
-   in Diff - in - Diff, all significant at 0.01, City significant at 0.05.<br>
-   As Barcelona is popular city, in spring to summer, have 0.046 upper sentiment than Florence, we suggest this.<br>
-   For all City: after event happened, sentiment showing a 0.189 increasing trend.<br>

#### Summer to Autumn: event on August

In [ ]:
sum_results1, sum_results2, sum_results3,_ = apply_diff_in_diff( summer_monthly_sentiment)

stargazer = Stargazer([sum_results1, sum_results2, sum_results3])
HTML(stargazer.render_html())

Base: Florence = 0, Bacelona = 1<br>

In Summer to Autumn season. August is event.<br>

Coefficient analysis: most significant at 0.01<br>
Event Treatment: showing a 0.079 increasing trend over all reviews sentiment. But not significant<br>
City: -0.156 showing a decreasing trend compare Barcelona to Florence. Significant<br>
Diff-in-Diff estimator: Showing a -0.167 decreasing trend for Barcelona, after event happened.<br>
-   in Diff - in - Diff, all significant at 0.01.<br>
-   As Barcelona is popular city, in summer to august, have 0.064 upper sentiment than Florence, we suggest this.<br>
-   For all City: after event happened, sentiment showing a 0.177 increasing trend.<br>

#### Autumn to Fall (Beginning of winter): event on December

In [ ]:
fall_results1, fall_results2, sumfall_results3,_= apply_diff_in_diff(fall_monthly_sentiment)

stargazer = Stargazer([fall_results1, fall_results2, sumfall_results3])
HTML(stargazer.render_html())

Base: Florence = 0, Bacelona = 1<br>

In Autumn to Winter season. December is event.<br>

Coefficient analysis: most significant at 0.01<br>
Event Treatment: this time significant at 0.05, this is we kind of expect, as sentiment should more influenced in winter season change. showing a 0.079 increasing trend over all reviews sentiment.<br>
City: -0.101 showing a decreasing trend compare Barcelona to Florence. Significant. Slight lower than previous, might Florence is also influenced little by winter season changes.<br>
Diff-in-Diff estimator: Showing a -0.173 decreasing trend for Barcelona, after event happened. We suggest in winter season changes, visotrs reviews' sentiment is influenced more than other seasons, although influence degree is slightly<br>
-   in Diff - in - Diff, all significant at 0.01.<br>
-   As Barcelona is popular city, in autumn to winter, have 0.069 upper sentiment than Florence, we suggest this.<br>
-   For all City: after event happened, sentiment showing a 0.169 increasing trend.<br>

### 2. Feature Engineering Extraction

In [16]:
#### nlp component setting #####
porter = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
# Load spaCy model
sp = spacy.load('en_core_web_sm')

mod = 0

def remove_digits(token):
    # Use regex to remove digits
    token_without_digits = re.sub(r'\d+', '', token)
    return token_without_digits

def abbr_or_lower(word):
    if re.match('([A-Z]+[a-z]*){2,}', word):
        return word
    else:
        return word.lower()
    
def tokenize(text, modulation):
    # Define regex for emojis and "!"
    emoji_exclam_pattern = re.compile(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251!]')
    
    tokens = []

    cleaned_text = emoji_exclam_pattern.sub(r'', text)
    if modulation < 2:
        for token in re.split(r'\W+', cleaned_text):
            token = remove_digits(token) # remove number
            if token is '': continue
            token = abbr_or_lower(token)
            if token and re.search('[a-zA-Z]', token):
                if modulation == 0:
                    tokens.append(token)
                elif modulation == 1 and token not in stop_words and porter: # remove stop words + stem
                    tokens.append(porter.stem(token))
        
        # Handle emojis and "!" separately to avoid removing them
        emoji_exclam_tokens = emoji_exclam_pattern.findall(text)
        for token in emoji_exclam_tokens:
            if token == '!':
                tokens.append(token)
            elif modulation < 2 and token in stop_words:
                continue  # If an emoji is somehow in stop_words, skip it
            else:
                tokens.append(token)
        
    else:
        # Handling for modulation >= 2, potentially using spaCy or other logic
        pass  # Implement as necessary
    
    return tokens

def custom_tokenizer(text):
    tokens = tokenize(text, modulation=mod)
    return tokens

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Filter where Is_Commented is True
commented_df = df_copy[df_copy['Is_Commented'] == True]

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer,ngram_range=(1, 2), max_df=0.25, min_df=0.0036)

# Fit and transform the cleaned text data
tfidf_features = tfidf_vectorizer.fit_transform(commented_df['Review_Body'])

# Convert to dense matrix (if needed)
tfidf_features_dense = tfidf_features.toarray()

# Get feature names
feature_names = tfidf_vectorizer.get_feature_names_out()

In [57]:
import numpy as np

# Sum tf-idf scores for each term
term_scores = np.sum(tfidf_features_dense, axis=0)

# Create a dictionary to store terms and their scores
term_score_dict = {term: score for term, score in zip(feature_names, term_scores)}

# Sort terms by their scores in descending order
sorted_terms = sorted(term_score_dict.items(), key=lambda x: x[1], reverse=True)

# Print the top 100 terms with their scores
for term, score in sorted_terms[:1000]:
    print(f"{term}: {score}")
# Print the top 100 terms with their scores
for term, score in sorted_terms[-1000:]:
    print(f"{term}: {score}")
print("\n\nLength of terms: ",len(term_score_dict))

!: 208.5133599625889
i: 146.88396164684627
we: 134.98366864248985
you: 133.28580827053864
for: 125.48366904324654
guide: 113.30620062991953
! !: 109.10506207155014
experience: 103.39006092791263
with: 102.18714638318855
great: 102.1686019174325
not: 98.0760833983548
on: 95.94049860241137
tour: 95.83976123429974
it was: 87.2531009571106
good: 86.57471641745074
beautiful: 86.5200126772142
but: 85.04733044081163
time: 83.7942249027614
this: 82.61419228653921
of the: 82.2670213091322
that: 82.11479336368424
at: 80.80355557114225
visit: 80.1113128005455
park: 76.76724019305568
there: 76.63475052708145
were: 75.1030584181821
t: 74.77859124355027
s: 73.47772929473666
an: 70.48270404818841
have: 70.43643362002209
nice: 69.92047641169019
see: 69.60602273608733
be: 66.15946035294901
worth: 64.63411132106852
are: 63.408388544782625
recommend: 63.3869998090347
tickets: 61.78370779994172
so: 61.66293249661133
all: 61.19857898511576
as: 59.58760079337982
amazing: 58.55791400563874
everything: 57.880

As tokenizing in reviews is large processing, and it's not usual to have a term with global large score. So we set max_df with very samll value = 0.25 or 0.3 to filter out terms. 
We checked with max_df = 0.6, 0.5, 0.4. Only stop words appears( we keep stop words to keep sentiment words like(no, not)): 257.35129992135427, and: 253.94346251422064, a: 221.65341402759285<br>

Untill we set max_df from 0.25 to 0.3, Order of terms in extracted by tf-idf is still same. '!' is top 1 important, imply that visitors like to use '!' to express experience of trip. Also, most of top appearance of terms are positive words. !: 207.78350533123077, ! !: 108.72901240734484, nice: 69.01308553961721. 

emojs are kind of less appearance over all reviews, are these emojs are kind of important compare all terms:👍: 15.276413828211869, 😊: 5.695564932529107, ❤: 7.05564773544313

#### LDA - topic modelling

In [ ]:
'''from collections.abc import Mapping
from gensim import corpora, models
import pprint
'''

In [ ]:
'''#!pip install gensim
!pip install --use-pep517 gensim==3.8.0'''